In [1]:
from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd


In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet'
sg = Subgrounds()
endpoint = endpoint
subgraph = sg.load_subgraph(endpoint)

CONFIG_COMMON_SWAPS_TOKENS = {
    "ethereum": {
        # [GHO]
        'search_tokens': ["0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f", ],
    }
}


In [6]:
# Build the query
chain = 'ethereum'
config = CONFIG_COMMON_SWAPS_TOKENS[chain]
last_materialized_timestamp = 1693362587
last_materialized_block = 18024449


tokens_where = []
tokens_where.append({
    'tokenBought_in': config['search_tokens'],
    'timestamp_gt': last_materialized_timestamp
})
tokens_where.append({
    'tokenBought_in': config['search_tokens'],
    'timestamp_gt': last_materialized_timestamp
})

swaps_data = pd.DataFrame()
for w in tokens_where:
    swaps = subgraph.Query.swapEvents(
        where=w,
        first=1000000
    )

    field_paths = [
        swaps.id,
        swaps.tx,
        swaps.timestamp,
        swaps.block,
        swaps.pool.address,
        swaps.buyer,
        swaps.tokenSold,
        swaps.tokenBought,
        swaps.amountSold,
        swaps.amountBought,
        swaps.amountSoldUSD,
        swaps.amountBoughtUSD,
    ]

    part = sg.query_df(field_paths, pagination_strategy=ShallowStrategy)
    swaps_data = pd.concat([swaps_data, part])

# swaps_data
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 15
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   swapEvents_id               32 non-null     object 
 1   swapEvents_tx               32 non-null     object 
 2   swapEvents_timestamp        32 non-null     int64  
 3   swapEvents_block            32 non-null     int64  
 4   swapEvents_pool_address     32 non-null     object 
 5   swapEvents_buyer            32 non-null     object 
 6   swapEvents_tokenSold        32 non-null     object 
 7   swapEvents_tokenBought      32 non-null     object 
 8   swapEvents_amountSold       32 non-null     float64
 9   swapEvents_amountBought     32 non-null     float64
 10  swapEvents_amountSoldUSD    32 non-null     float64
 11  swapEvents_amountBoughtUSD  32 non-null     float64
dtypes: float64(4), int64(2), object(6)
memory usage: 3.2+ KB


In [9]:
cp = swaps_data.copy()


In [15]:
swaps_data = cp.copy()
swaps_data.rename(columns={
    'swapEvents_tx': 'tx_hash',
    'swapEvents_timestamp': 'block_unixtimestamp',
    'swapEvents_block': 'block_number',
    'swapEvents_pool_address': 'pool_address',
    'swapEvents_buyer': 'recipient_address',
    'swapEvents_tokenSold': 'token_in_address',
    'swapEvents_tokenBought': 'token_out_address',
    'swapEvents_amountSold': 'token_in_amount',
    'swapEvents_amountBought': 'token_out_amount',
    'swapEvents_amountSoldUSD': 'token_in_amount_usd',
    'swapEvents_amountBoughtUSD': 'token_out_amount_usd',
}, inplace=True)

swaps_data['block_timestamp'] = pd.to_datetime(
    swaps_data['block_unixtimestamp'], unit='s')
swaps_data['chain'] = chain
swaps_data['platform'] = 'curve'
swaps_data['is_aggregator'] = False
# swaps_data['recipient_address'] = swaps_data.sender_address
swaps_data['sender_address'] = None
swaps_data['event_index'] = None
swaps_data['caller_address'] = None
swaps_data['token_in_symbol'] = None
swaps_data['token_out_symbol'] = None
swaps_data['pool_id'] = swaps_data.pool_address
swaps_data['fee_tier'] = None
# swaps_data['event_index'] = swaps_data.event_index.astype(int)
# swaps_data['event_index'] = swaps_data.apply(
#     lambda row: row['swaps_id'].replace(row['tx_hash'], ''), axis=1)


swaps_data['token_in_price_usd'] = swaps_data.token_in_amount_usd / \
    swaps_data.token_in_amount
swaps_data['token_out_price_usd'] = swaps_data.token_out_amount_usd / \
    swaps_data.token_out_amount
# swaps_data['block_number'] = None

swaps_data = swaps_data[[
    "tx_hash",
    "event_index",
    "block_timestamp",
    "block_unixtimestamp",
    "block_number",
    "chain",
    "platform",
    "pool_address",
    "pool_id",
    "fee_tier",
    "is_aggregator",
    "caller_address",
    "sender_address",
    "recipient_address",
    "token_in_address",
    "token_out_address",
    "token_in_symbol",
    "token_out_symbol",
    "token_in_amount",
    "token_in_amount_usd",
    "token_in_price_usd",
    "token_out_amount",
    "token_out_amount_usd",
    "token_out_price_usd"
]]

# swaps_data.sort_values(by=['block_number', 'event_index'], inplace=True)
# last_block = swaps_data.block_number.max()
# last_timestamp = swaps_data.block_unixtimestamp.max()


KeyError: "['pool_id', 'fee_tier'] not in index"

In [13]:
swaps_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 15
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tx_hash               32 non-null     object        
 1   event_index           0 non-null      object        
 2   block_timestamp       32 non-null     datetime64[ns]
 3   block_unixtimestamp   32 non-null     int64         
 4   block_number          32 non-null     int64         
 5   chain                 32 non-null     object        
 6   platform              32 non-null     object        
 7   pool_address          32 non-null     object        
 8   is_aggregator         32 non-null     bool          
 9   caller_address        0 non-null      object        
 10  recipient_address     32 non-null     object        
 11  token_in_address      32 non-null     object        
 12  token_out_address     32 non-null     object        
 13  token_in_symbol       

In [14]:
swaps_data


,tx_hash,event_index,block_timestamp,block_unixtimestamp,block_number,chain,platform,pool_address,is_aggregator,caller_address,...,token_in_address,token_out_address,token_in_symbol,token_out_symbol,token_in_amount,token_in_amount_usd,token_in_price_usd,token_out_amount,token_out_amount_usd,token_out_price_usd
0,0x04aa54dbed6d2804102ee49b37325eebe18053bad7ba...,None,2023-08-31 18:08:11,1693505291,18036248,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,3167.337130,3244.505402,1.024364,3231.624925,3160.888150,0.978111
1,0x0c7bb972dfef8f6f58844698ac3b86f11ecae37afcfd...,None,2023-09-05 02:46:59,1693882019,18067390,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,2647.879585,2733.834299,1.032462,2722.716476,2641.308279,0.970100
2,0x0fce21a233929243000a9be728d10b9ee52b5201e9c0...,None,2023-09-12 22:45:23,1694558723,18123307,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,2401.717514,2494.651999,1.038695,2482.859811,2389.720885,0.962487
3,0x14e6eae08fa2dadda7430bb084fe4648867b9ca3b2ac...,None,2023-09-07 00:07:23,1694045243,18080871,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,2317.394295,2382.890696,1.028263,2391.681744,2311.379843,0.966425
4,0x2cc968a59ba555b0183b87f4a0e3df1f00b5a7852f5a...,None,2023-09-11 15:43:35,1694447015,18114078,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,3702.177386,3835.134777,1.035913,3845.734904,3701.450267,0.962482
5,0x2fd1ccbf8bf9f40a5e248bb57eb7c953c316231223a3...,None,2023-09-19 16:28:47,1695140927,18171150,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,5117.132523,5259.429475,1.027808,5239.026493,5042.496146,0.962487
6,0x635988bb38679551b3f7fdbdebc416c6a30ee6864c5c...,None,2023-08-31 09:30:47,1693474247,18033673,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,4443.678379,4526.143541,1.018558,4551.943136,4452.306023,0.978111
7,0x67a0495935ce17e10483e697a57374c7d34b52e6f5dd...,None,2023-09-03 18:53:47,1693767227,18057879,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,2626.123910,2681.374749,1.021039,2711.372398,2630.303385,0.970100
8,0x6b5232c5045d340eeaf549b2fadba76265d44b90d77b...,None,2023-09-02 10:27:35,1693650455,18048229,ethereum,curve,0x86152df0a0e321afb3b0b9c4deb813184f365ada,False,None,...,0xf939e0a03fb07f59a73314e73794be0e57ac1b4e,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,441.204481,450.159658,1.020297,450.486951,441.290733,0.979586
9,0x6eefc507a4522ade022905daf16ce99242650a2e18b5...,None,2023-09-21 07:28:59,1695281339,18182768,ethereum,curve,0xbc90fec043e6df6a084e18df9435ee037c940b2d,False,None,...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,None,None,16.436988,16.438633,1.000100,16.991954,16.563060,0.974759


In [61]:
l = int(swaps_data.block_number.max())
